## About Dataset
Context
This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.
Columns

- asin - ID of the product, like B000FA64PK
- helpful - helpfulness rating of the review - example: 2/3.
- overall - rating of the product.
- reviewText - text of the review (heading).
- reviewTime - time of the review (raw).
- reviewerID - ID of the reviewer, like A3SPTOKDG7WBLN
- reviewerName - name of the reviewer.
- summary - summary of the review (description).
- unixReviewTime - unix timestamp.

Acknowledgements
This dataset is taken from Amazon product data, Julian McAuley, UCSD website. http://jmcauley.ucsd.edu/data/amazon/

License to the data files belong to them.

Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).

#### Best Practises
1. Preprocessing And Cleaning
2. Train Test Split
3. BOW,TFIDF,Word2vec
4. Train ML algorithms

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [157]:
# loading the dataset
df=pd.read_csv('all_kindle_review.csv')

In [158]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [159]:
df=df[['reviewText','rating']]

In [8]:
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [11]:
# checking for missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [10]:
df.shape

(12000, 2)

In [13]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

### Preprocessing and Cleaning

In [160]:
# postive review is 0 and negative review is 1
df['rating']=df['rating'].apply(lambda x:1 if x<3 else 0)

In [142]:
df.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",0,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",0,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",0,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",0,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",0,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [161]:
df['rating'].unique()

array([0, 1], dtype=int64)

In [18]:
df['rating'].value_counts()

rating
0    8000
1    4000
Name: count, dtype: int64

In [162]:
# lower case
df['reviewText']=df['reviewText'].str.lower()

In [28]:
import re
import nltk 
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from bs4 import BeautifulSoup

In [163]:
# removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]+','',x))

#### Example:
```plaintext
                  reviewText                              cleanText
0          <p>This is <b>great</b>!</p>                 This is great!
1  <div>Awesome <a href='url'>product</a>!</div>       Awesome product!
2    <span>Highly <i>recommended</i>.</span>          Highly recommended.
```

In [146]:
df['reviewText']

0        jace rankin may be short but hes nothing to me...
1        great short read  i didnt want to put it down ...
2        ill start by saying this is the first of four ...
3        aggie is angela lansbury who carries pocketboo...
4        i did not expect this type of book to be in li...
                               ...                        
11995    valentine cupid is a vampire jena and ian anot...
11996    i have read all seven books in this series apo...
11997    this book really just wasnt my cuppa  the situ...
11998    tried to use it to charge my kindle it didnt e...
11999    taking instruction is a look into the often hi...
Name: reviewText, Length: 12000, dtype: object

In [147]:
# removing stopwords
stop_words=set(stopwords.words('english'))

In [164]:
df['reviewText']=df['reviewText'].apply(lambda x:' '.join([word for word in x.split()
                                                           if word not in stop_words]))

In [38]:
# remove URL
df['reviewText']

0        jace rankin may short hes nothing mess man hau...
1        great short read didnt want put read one sitti...
2        ill start saying first four books wasnt expect...
3        aggie angela lansbury carries pocketbooks inst...
4        expect type book library pleased find price right
                               ...                        
11995    valentine cupid vampire jena ian another vampi...
11996    read seven books series apocalypticadventure o...
11997    book really wasnt cuppa situation man capturin...
11998    tried use charge kindle didnt even register ch...
11999    taking instruction look often hidden world sex...
Name: reviewText, Length: 12000, dtype: object

In [165]:
df['reviewText']=df['reviewText'].apply(lambda x:
                 re.sub(r'(?:http|https|ftp|ssh)://[\w.-]+(?:\.[a-zA-Z]{2,})(?:[/?#]\S*)?','',str(x)))

### Optimized Regular Expression:
```regex
(?:http|https|ftp|ssh)://[\w.-]+(?:\.[a-zA-Z]{2,})(?:[/?#]\S*)?
```

---

### Breaking It Down with Examples:

#### 1. **Protocol Matching**: `(?:http|https|ftp|ssh)://`
- **Matches**:
  - `http://`
  - `https://`
  - `ftp://`
  - `ssh://`
- **Explanation**:
  - The group `(?: ... )` is **non-capturing**, meaning it does not save the matched content but ensures it matches one of the protocols (`http`, `https`, `ftp`, `ssh`).
  - Followed by `://`, which is a literal match of the standard URL separator.

---

#### 2. **Domain and Subdomain**: `[\w.-]+`
- **Matches**:
  - `example`
  - `sub.example`
  - `my-site.example123`
- **Explanation**:
  - `[\w.-]+`: Matches one or more characters that are:
    - `\w`: Alphanumeric characters (letters and digits) or underscores (`_`).
    - `.`: Periods (for subdomains).
    - `-`: Hyphens (for domain names).
- **Example URLs**:
  - `http://example.com`
  - `https://sub.domain.org`
  - `ftp://my-site.example123`

---

#### 3. **Top-Level Domain (TLD)**: `(?:\.[a-zA-Z]{2,})`
- **Matches**:
  - `.com`
  - `.org`
  - `.co.uk`
- **Explanation**:
  - `(?: ... )`: Non-capturing group to match the TLD.
  - `\.`: Matches a literal dot (`.`).
  - `[a-zA-Z]{2,}`: Matches a string of 2 or more alphabetic characters (e.g., `com`, `org`).
- **Example URLs**:
  - `http://example.com`
  - `https://domain.co.uk`
  - `ftp://site.org`

---

#### 4. **Optional Path, Query, or Fragment**: `(?:[/?#]\S*)?`
- **Matches**:
  - `/path/to/resource`
  - `?query=value`
  - `#fragment`
- **Explanation**:
  - `(?: ... )?`: Optional group to match the path, query, or fragment.
  - `[/?#]`: Matches a single character that starts a path (`/`), query (`?`), or fragment (`#`).
  - `\S*`: Matches zero or more non-whitespace characters (to capture everything after the separator).
- **Example URLs**:
  - `https://example.com/path/to/resource`
  - `http://example.com?query=value`
  - `https://example.com#fragment`


In [150]:
df['reviewText'].sample().values

array(['boring word book needs describe interesting description amazing reviews amazon thought would give try big mistake took month half read would read 5 pages literally fall asleep extremely long time read book usually read book one sitting reason finished reading think order appropriately write review need read whole bookthe majority book would appropriate child felt like author didnt tackle big topics make fall love book author let connect characters rendering useless could live die fall love wouldnt care love book makes feel something whether happy sad book made feel angry wasting timeclearly would recommend book especially fan ya book wont cut'],
      dtype=object)

In [151]:
# lemmatization
from nltk.stem import WordNetLemmatizer

In [152]:
lemma=WordNetLemmatizer()

In [166]:
df['reviewText']=df['reviewText'].apply(lambda x:' '.join(lemma.lemmatize(word) for word in x.split()))

In [167]:
X=df['reviewText']
y=df['rating']

### Train Test Split

In [87]:
from sklearn.model_selection import train_test_split

In [101]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=.20,random_state=42)

### Bag Of Words

In [102]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [103]:
bow=CountVectorizer()

In [104]:
xtrain_bow=bow.fit_transform(xtrain).toarray()
xtest_bow=bow.transform(xtest).toarray()

# TF-IDF

In [105]:
tfidf=TfidfVectorizer()

In [106]:
xtrain_tfidf=tfidf.fit_transform(xtrain).toarray()
xtest_tfidf=tfidf.transform(xtest).toarray()

In [108]:
from sklearn.naive_bayes import GaussianNB

In [110]:
gnb_bow=GaussianNB().fit(xtrain_bow,ytrain)

In [111]:
gnb_tfidf=GaussianNB().fit(xtrain_tfidf,ytrain)

In [112]:
from sklearn.metrics import accuracy_score,classification_report

In [113]:
ypred_bow=gnb_bow.predict(xtest_bow)

In [114]:
ypred_tfidf=gnb_tfidf.predict(xtest_tfidf)

#### BOW metrics

In [115]:
accuracy_score(ytest,ypred_bow)

0.5775

In [118]:
print(classification_report(ytest,ypred_bow))

              precision    recall  f1-score   support

           0       0.74      0.56      0.64      1597
           1       0.41      0.62      0.50       803

    accuracy                           0.58      2400
   macro avg       0.58      0.59      0.57      2400
weighted avg       0.63      0.58      0.59      2400



#### TFIDF - metrics

In [116]:
accuracy_score(ytest,ypred_tfidf)

0.57875

In [119]:
print(classification_report(ytest,ypred_tfidf))

              precision    recall  f1-score   support

           0       0.74      0.57      0.64      1597
           1       0.41      0.60      0.49       803

    accuracy                           0.58      2400
   macro avg       0.57      0.58      0.57      2400
weighted avg       0.63      0.58      0.59      2400



# Word2Vec

In [120]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
import gensim.downloader as api

In [121]:
wv=api.load('word2vec-google-news-300')

In [122]:
wv['cricket']

array([-3.67187500e-01, -1.21582031e-01,  2.85156250e-01,  8.15429688e-02,
        3.19824219e-02, -3.19824219e-02,  1.34765625e-01, -2.73437500e-01,
        9.46044922e-03, -1.07421875e-01,  2.48046875e-01, -6.05468750e-01,
        5.02929688e-02,  2.98828125e-01,  9.57031250e-02,  1.39648438e-01,
       -5.41992188e-02,  2.91015625e-01,  2.85156250e-01,  1.51367188e-01,
       -2.89062500e-01, -3.46679688e-02,  1.81884766e-02, -3.92578125e-01,
        2.46093750e-01,  2.51953125e-01, -9.86328125e-02,  3.22265625e-01,
        4.49218750e-01, -1.36718750e-01, -2.34375000e-01,  4.12597656e-02,
       -2.15820312e-01,  1.69921875e-01,  2.56347656e-02,  1.50146484e-02,
       -3.75976562e-02,  6.95800781e-03,  4.00390625e-01,  2.09960938e-01,
        1.17675781e-01, -4.19921875e-02,  2.34375000e-01,  2.03125000e-01,
       -1.86523438e-01, -2.46093750e-01,  3.12500000e-01, -2.59765625e-01,
       -1.06933594e-01,  1.04003906e-01, -1.79687500e-01,  5.71289062e-02,
       -7.41577148e-03, -

In [125]:
df['reviewText'][0]

'jace rankin may short he nothing mess man hauled saloon undertaker know he famous bounty hunter oregon 1890s shot man saloon finished year long quest avenge sister murder trying figure next snottynosed farm boy rescued gang bully offer money kill man forced ranch reluctantly agrees bring man justice kill outright first need tell sister widower newskyla kyle springer bailey riding trail sleeping ground past month trying find jace want revenge man killed husband took ranch amongst crime shes keen detour jace want take realizes shes option hide behind boy persona best try keep pace confrontation along way get shot jace discovers kyles kyla come clean whole reason need scoundrel dead hope hell still help herthe book share touching moment slowblooming romance kyla find good reason fear men hide behind boy persona watching jace slowly pull shell help conquer fear endearing pain real deeplyrooted didnt disappear face sexiness neither understandable aversion marriage magically disappear round

In [168]:
df1=df.copy()

In [169]:
df1.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,0
1,great short read didnt want put read one sitti...,0
2,ill start saying first four book wasnt expecti...,0
3,aggie angela lansbury carry pocketbook instead...,0
4,expect type book library pleased find price right,0


In [195]:
words=[]
for i in range(len(df1)):
    review=df1['reviewText'][i].split()
    words.append(review)

In [196]:
words

[['jace',
  'rankin',
  'may',
  'short',
  'he',
  'nothing',
  'mess',
  'man',
  'hauled',
  'saloon',
  'undertaker',
  'know',
  'he',
  'famous',
  'bounty',
  'hunter',
  'oregon',
  '1890s',
  'shot',
  'man',
  'saloon',
  'finished',
  'year',
  'long',
  'quest',
  'avenge',
  'sister',
  'murder',
  'trying',
  'figure',
  'next',
  'snottynosed',
  'farm',
  'boy',
  'rescued',
  'gang',
  'bully',
  'offer',
  'money',
  'kill',
  'man',
  'forced',
  'ranch',
  'reluctantly',
  'agrees',
  'bring',
  'man',
  'justice',
  'kill',
  'outright',
  'first',
  'need',
  'tell',
  'sister',
  'widower',
  'newskyla',
  'kyle',
  'springer',
  'bailey',
  'riding',
  'trail',
  'sleeping',
  'ground',
  'past',
  'month',
  'trying',
  'find',
  'jace',
  'want',
  'revenge',
  'man',
  'killed',
  'husband',
  'took',
  'ranch',
  'amongst',
  'crime',
  'shes',
  'keen',
  'detour',
  'jace',
  'want',
  'take',
  'realizes',
  'shes',
  'option',
  'hide',
  'behind',
  'bo

In [198]:
model = Word2Vec(vector_size=100, window=5, min_count=5)

In [199]:
model.build_vocab(words)

In [200]:
model.train(words, total_examples=model.corpus_count, epochs=model.epochs)

(2757980, 3277155)

In [217]:
# To get all the vocabulary
model.wv.index_to_key

['book',
 'story',
 'read',
 'one',
 'character',
 'like',
 'good',
 'would',
 'really',
 'love',
 'time',
 'get',
 'author',
 'reading',
 'series',
 'much',
 'first',
 'well',
 'even',
 'didnt',
 'short',
 'know',
 'could',
 'way',
 'great',
 'make',
 'sex',
 'little',
 'dont',
 'thing',
 'want',
 'two',
 'think',
 'find',
 'plot',
 'also',
 'romance',
 'end',
 'life',
 'im',
 'see',
 'enjoyed',
 'go',
 'scene',
 'never',
 'take',
 'many',
 'woman',
 'lot',
 'written',
 'say',
 'kindle',
 'work',
 'thought',
 'bit',
 'found',
 'going',
 'give',
 'year',
 'interesting',
 'liked',
 'writing',
 'novel',
 'loved',
 'another',
 'feel',
 'got',
 'better',
 'come',
 'man',
 'hot',
 'still',
 'enough',
 'back',
 'though',
 'people',
 'reader',
 'made',
 'review',
 'something',
 'part',
 'friend',
 'star',
 'cant',
 'page',
 'need',
 'bad',
 'keep',
 'new',
 'wasnt',
 'free',
 'doesnt',
 'world',
 'relationship',
 'enjoy',
 'recommend',
 'together',
 'next',
 'start',
 'felt',
 'best',
 'put',

In [218]:
# vocabulary size
model.corpus_count

12000

In [219]:
model.wv.similar_by_word('good')

[('decent', 0.8474252820014954),
 ('soso', 0.8323930501937866),
 ('easy', 0.8274370431900024),
 ('great', 0.8187395334243774),
 ('ok', 0.8168004751205444),
 ('nice', 0.8164852857589722),
 ('satisfying', 0.8115332126617432),
 ('okay', 0.7997053861618042),
 ('entertaining', 0.7958430647850037),
 ('line', 0.7951013445854187)]

In [203]:
def avg_word2vec(doc):
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

In [204]:
X=[]
for i in range(len(words)):
    X.append(avg_word2vec(words[i]))

In [205]:
X

[array([-0.00451106,  0.32788548,  0.18290296,  0.11370344, -0.0538342 ,
        -0.47346297,  0.04133103,  0.7709347 , -0.2121683 , -0.07910848,
        -0.17233677, -0.34462535,  0.05582429,  0.13465774, -0.0311993 ,
        -0.20770647,  0.09389907, -0.49673954,  0.12483352, -0.41115332,
         0.16311215,  0.08299365,  0.08790953, -0.07114077, -0.06399927,
         0.00891875, -0.34904474,  0.08004034, -0.2741799 ,  0.13836972,
         0.3531873 , -0.14168966,  0.08080612, -0.27169535, -0.08097074,
         0.3926202 ,  0.01206633, -0.35876542, -0.17497164, -0.6368484 ,
        -0.00765288, -0.3594032 , -0.04394507,  0.0043478 ,  0.15359522,
        -0.23634647, -0.02617603, -0.04466194,  0.1952758 ,  0.14544213,
         0.05765164, -0.17380923, -0.13303684,  0.0625059 , -0.22118585,
         0.18302049,  0.11273525,  0.0958912 , -0.3329677 , -0.0726212 ,
         0.00170759,  0.08856918, -0.03063617, -0.01327976, -0.29878634,
         0.362387  ,  0.24852607,  0.37127304, -0.4

In [206]:
len(X)

12000

In [207]:
X=pd.DataFrame(X)

In [209]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=.20,random_state=42)

In [210]:
from sklearn.ensemble import RandomForestClassifier

In [211]:
rf=RandomForestClassifier()

In [212]:
rf.fit(xtrain,ytrain)

RandomForestClassifier()

In [213]:
ypred=rf.predict(xtest)

In [214]:
from sklearn.metrics import accuracy_score,classification_report

In [215]:
accuracy_score(ytest,ypred)

0.7675

In [220]:
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.79      0.88      0.83      1597
           1       0.70      0.54      0.61       803

    accuracy                           0.77      2400
   macro avg       0.74      0.71      0.72      2400
weighted avg       0.76      0.77      0.76      2400

